In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

website = "https://yokohama-atg.com/usa/tire-selector-yokohama-off-highway-tires/"
path = "C:\\Users\\cheta\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"  # Change according to your PC's file path
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

my_designs = ['350']

folder = 'Alliance'

if not os.path.exists(folder):
    os.mkdir(folder)

try:
    driver.get(website)

    # Wait for the cookie accept button and click it
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[@data-cookie-set="accept"]'))
        )
        accept_button.click()
    except Exception as e:
        print(f"Cookie notice not found or could not be clicked: {str(e)}")

    print('working...')
    
    # Click on the brand select button
    select_buttons = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//span[@class="select2-arrow"]'))
    )
    brands_button = select_buttons[2]
    brands_button.click()

    # Select the brand
    select_brands = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//li[@class="select2-results-dept-0 select2-result select2-result-selectable"]'))
    )
    select_brands[0].click()

    # Wait for the loader to disappear
    WebDriverWait(driver, 10).until(
        EC.invisibility_of_element((By.XPATH, '//div[@class="loader-wrapper"]'))
    )

    # Click on the design select button
    select_buttons = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//span[@class="select2-arrow"]'))
    )
    design_button = select_buttons[3]
    design_button.click()

    select_design = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//li[@class="select2-results-dept-0 select2-result select2-result-selectable"]'))
    )

    def get_design(values):
        new_design = []

        if isinstance(values, str):
            values = [values]

        for des in select_design:
            if des.text in values:
                new_design.append(des)
        return new_design

    designs = get_design(my_designs)
    design = designs[0]

    print(f'For Design: {design.text}')
    design.click()

    # Click the search button
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@title="Search"]'))
    )
    search_button.click()
    print('Search...')

    # Wait for tables to become visible
    try:
        # Find the parent div element containing the tables you're interested in
        data_in_mm = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="data_in_mm "]'))
        )

        # Find all tables within the parent div
        tables_mm = data_in_mm.find_elements(By.XPATH, './/table[@id="tblResultView"]')
        headers = []
        tables_in_mm = []

        for i, table in enumerate(tables_mm):
            # Extract header row
            header_row_mm = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'tr')
            headers_heads = {index: heads.find_elements(By.TAG_NAME, 'th') for index, heads in enumerate(header_row_mm) if index!=3}

            headers_heads_texts = {
                index: {i: th.text for i, th in enumerate(th_list)}
                for index, th_list in headers_heads.items()
            }

            tables_in_mm.append({f"Table_{i+1}": headers_heads_texts})
            print('Header Collected')
        
        headers.append({"Tables_in_mm": tables_in_mm})

        data_in_inch = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="data_in_inch "]'))
        )

        # Find all table elements within the parent div
        tables_inch = data_in_inch.find_elements(By.XPATH, './/table[@id="tblResultView"]')

        # Execute JavaScript to change the display style to block (making it visible)
        driver.execute_script("arguments[0].style.display = 'block';", data_in_inch)

    except Exception as e:
        print(f"An error occurred while fetching tables 'tblResultView': {str(e)}")

finally:
    time.sleep(5)
    driver.quit()
    print("All data has been collected and the browser is closed.")

working...
For Design: 350
Search...
Header Collected
Header Collected
All data has been collected and the browser is closed.


In [49]:
table1 = headers[0]['Tables_in_mm'][0]['Table_1']

In [50]:
table1

{0: {0: 'Size',
  1: 'Rim',
  2: 'Unloaded Dimension',
  3: 'Loaded Static radius\nmm',
  4: 'Rolling Circumference\nmm',
  5: 'Load Index\nPR\nSymbols',
  6: 'Inflation Pressure\nbar',
  7: 'Recommended Load'},
 1: {0: 'Speed'},
 2: {0: 'Not high and sustained torque;Road transport'},
 4: {0: 'SW\nmm',
  1: 'OD\nmm',
  2: '10\nkmph',
  3: '20\nkmph',
  4: '25\nkmph',
  5: '30\nkmph',
  6: '40\nkmph',
  7: '50\nkmph',
  8: '65\nkmph'}}

In [52]:
table1[0][0]

'Size'

In [59]:
import pandas as pd

column_names = [table1[0][i] for i in range(len(table1[0]))]

data_dict = {col_name: '' for col_name in column_names}

df = pd.DataFrame([data_dict] * 5)

df

,Size,Rim,Unloaded Dimension,Loaded Static radius\nmm,Rolling Circumference\nmm,Load Index\nPR\nSymbols,Inflation Pressure\nbar,Recommended Load
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
